In [1]:
import numpy as np
import pandas as pd
import requests
from time import sleep

In [2]:
sp_cep = pd.read_csv('ceps/sp.cepaberto.csv',names=['cep','rua','bairro','id_cidade','id_bairro'])
rs_cep = pd.read_csv('ceps/rs.cepaberto.csv',names=['cep','rua','bairro','id_cidade','id_bairro'])
mg_cep = pd.read_csv('ceps/mg.cepaberto.csv',names=['cep','rua','bairro','id_cidade','id_bairro'])
sc_cep = pd.read_csv('ceps/sc.cepaberto.csv',names=['cep','rua','bairro','id_cidade','id_bairro'])

cidades = pd.read_csv('ceps/cities.csv',names=['id_cidade','nome_cidade','id_estado'])
estados = pd.read_csv('ceps/states.csv',names=['id_estado','nome_estado','sigla_estado'])
cidades_estados = pd.merge(cidades,estados,how='left',on='id_estado')

#cria a base
sp_cep = sp_cep.merge(cidades_estados,how='left',on='id_cidade')
rs_cep = rs_cep.merge(cidades_estados,how='left',on='id_cidade')
mg_cep = mg_cep.merge(cidades_estados,how='left',on='id_cidade')
sc_cep = sc_cep.merge(cidades_estados,how='left',on='id_cidade')

In [3]:
sp_num_ceps = sp_cep.nome_cidade.value_counts().reset_index().rename(columns={'index':'nome_cidade','nome_cidade':'cidade_counts'})
rs_num_ceps = rs_cep.nome_cidade.value_counts().reset_index().rename(columns={'index':'nome_cidade','nome_cidade':'cidade_counts'})
mg_num_ceps = mg_cep.nome_cidade.value_counts().reset_index().rename(columns={'index':'nome_cidade','nome_cidade':'cidade_counts'})
sc_num_ceps = sc_cep.nome_cidade.value_counts().reset_index().rename(columns={'index':'nome_cidade','nome_cidade':'cidade_counts'})

In [4]:
sp_cep = sp_cep.merge(sp_num_ceps,how='left',on='nome_cidade')
rs_cep = rs_cep.merge(rs_num_ceps,how='left',on='nome_cidade')
mg_cep = mg_cep.merge(mg_num_ceps,how='left',on='nome_cidade')
sc_cep = sc_cep.merge(sc_num_ceps,how='left',on='nome_cidade')

In [5]:
sp_cep = sp_cep[sp_cep.cidade_counts > 100]
rs_cep = rs_cep[rs_cep.cidade_counts > 100]
mg_cep = mg_cep[mg_cep.cidade_counts > 100]
sc_cep = sc_cep[sc_cep.cidade_counts > 100]

In [6]:
fracao = 0.1
sp_cep = sp_cep.groupby('id_cidade', group_keys=False).apply(lambda x: x.sample(frac=fracao))
rs_cep = rs_cep.groupby('id_cidade', group_keys=False).apply(lambda x: x.sample(frac=fracao))
mg_cep = mg_cep.groupby('id_cidade', group_keys=False).apply(lambda x: x.sample(frac=fracao))
sc_cep = sc_cep.groupby('id_cidade', group_keys=False).apply(lambda x: x.sample(frac=fracao))


In [34]:
rs_bairros = rs_cep.groupby(['id_cidade']).apply(lambda df: df.sample(1))[['cep','id_cidade']].reset_index(drop=True)
sp_bairros = sp_cep.groupby(['id_cidade']).apply(lambda df: df.sample(1))[['cep','id_cidade']].reset_index(drop=True)
sc_bairros = sc_cep.groupby(['id_cidade']).apply(lambda df: df.sample(1))[['cep','id_cidade']].reset_index(drop=True)
mg_bairros = mg_cep.groupby(['id_cidade']).apply(lambda df: df.sample(1))[['cep','id_cidade']].reset_index(drop=True)

In [35]:
sp_bairros.iloc[6]

cep          7434525
id_cidade        709
Name: 6, dtype: int64

In [40]:
sp_bairros = sp_bairros[sp_bairros.cep != 7434525]

In [29]:
requests.get(base_url + str(07400200), headers={'Authorization': 'Token token={}'.format(token)})

SyntaxError: invalid token (<ipython-input-29-419ca439062c>, line 1)

In [41]:
base_url = 'http://www.cepaberto.com/api/v3/cep?cep='
token = 'fa20d2aaf33fcb06a6614915bca26afb'

df = sp_bairros
dumm = pd.DataFrame(columns=['cep','id_cidade','latitude','longitude'])
data_len = len(df)
for i in range(len(df)):
    cep = df.iloc[i]['cep']
    id_cidade = df.iloc[i]['id_cidade']
    
    url = base_url + str(cep)
    r = requests.get(url, headers={'Authorization': 'Token token={}'.format(token)})
    dumm = dumm.append(pd.DataFrame([[cep,id_cidade,r.json()['latitude'],r.json()['longitude']]],columns=['cep','id_cidade','latitude','longitude']))
    print('CEP: {}, LAT: {}, LONG: {}, PROG: {}'.format(cep, r.json()['latitude'], r.json()['longitude'],(i/data_len)*100))
    sleep(4.1)

CEP: 13465190, LAT: -22.7480569, LONG: -47.3344583, PROG: 0.0
CEP: 13903270, LAT: -22.7144109, LONG: -46.7699743, PROG: 0.9708737864077669
CEP: 16900453, LAT: -20.9098065, LONG: -51.3751202, PROG: 1.9417475728155338
CEP: 16018660, LAT: -21.2255897, LONG: -50.430327, PROG: 2.912621359223301
CEP: 14808355, LAT: -21.8115264, LONG: -48.1520784, PROG: 3.8834951456310676
CEP: 13601081, LAT: -22.3621008, LONG: -47.3855379, PROG: 4.854368932038835
CEP: 19813372, LAT: -22.6604341, LONG: -50.4187813, PROG: 5.825242718446602
CEP: 12940610, LAT: -23.1146494, LONG: -46.549607, PROG: 6.796116504854369
CEP: 18703778, LAT: -23.1225888, LONG: -48.9264086, PROG: 7.766990291262135
CEP: 14787130, LAT: -20.5270679, LONG: -48.5695587, PROG: 8.737864077669903


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
sp_cep = sp_cep.merge(dumm.drop('cep',axis=1),how='left',on=['id_cidade']).dropna(axis=0)
sp_cep = sp_cep.dropna(axis=0)

In [19]:
sp_cep.to_csv('ceps/sp_clean.csv',index=False)

In [ ]:
sp_cep.isnull().sum()

In [ ]:
sp_cep

In [ ]:
payload = {'SellerCEP': from_cep,
           'RecipientCEP': to_cep,
           'ShipmentInvoiceValue': 10,
           'RecipientCountry': 10,
           'ShippingItemArray': [{
                    "Weight": 2.1,
                    "Length": 14,
                    "Height": 20,
                    "Width": 15,
                    "Quantity": 1}],
          }

headers = {
  'Content-Type': 'application/json',
  'token': '1C987904R7FC6R4396R88A2R3ECF09B9282E'
}

r = requests.get('http://api.frenet.com.br/shipping/quote', params=payload, headers=headers)

In [18]:
from_cep = 13084778
to_cep = 90050100

In [32]:
base_url = 'https://www.melhorenvio.com.br/api/v2/calculator?'

payload = {'from': from_cep,
           'to': to_cep,
           'width': 10,
           'height': 10,
           'length': 10,
           'weight': 10,
           'receipt': False,
           'own_hand': False,
           'insurance_value': 20.50,
           'services': '1,2,3,4'}

In [33]:
r = requests.get('https://www.melhorenvio.com.br/api/v2/calculator', params=payload)

In [34]:
r.json()

[{'company': {'id': 1,
   'name': 'Correios',
   'picture': 'https://www.melhorenvio.com.br/images/shipping-companies/correios.png?v=2'},
  'error': 'Largura mínima do pacote é 11cm',
  'id': 1,
  'name': 'PAC'},
 {'company': {'id': 1,
   'name': 'Correios',
   'picture': 'https://www.melhorenvio.com.br/images/shipping-companies/correios.png?v=2'},
  'error': 'Largura mínima do pacote é 11cm',
  'id': 2,
  'name': 'SEDEX'},
 {'additional_services': {'collect': False,
   'own_hand': False,
   'receipt': False},
  'company': {'id': 2,
   'name': 'JadLog',
   'picture': 'https://www.melhorenvio.com.br/images/shipping-companies/jadlog.png?v=2'},
  'currency': 'R$',
  'delivery_time': 5,
  'discount': '9.05',
  'id': 3,
  'name': '.Package',
  'packages': [{'delivery_time': 5,
    'dimensions': {'height': 10, 'length': 10, 'width': 10},
    'discount': '9.05',
    'format': 'box',
    'insurance_value': '20.5',
    'price': '39.53',
    'products': [],
    'weight': 10}],
  'price': '39.53'